In [15]:
import os
import pandas as pd
import torch
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer


In [4]:
# ----------- Setting up the stage ----------
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
VECTOR_DB_DIR = os.path.join(DATA_DIR, "chroma_db") # Directory where ChromaDB is persisted

COLLECTION_NAME = "pakistan_laws_chunks_collection"
EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5" # This MUST be the exact model you used to generate embeddings

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Vector DB directory: {VECTOR_DB_DIR}")

Project root: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG
Data directory: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data
Vector DB directory: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data/chroma_db


In [5]:
# ----- Setting up my love with chromadb via its client ------
print(f"\nInitializing ChromaDB client at: {VECTOR_DB_DIR}")
try:
    client = chromadb.PersistentClient(path=VECTOR_DB_DIR)
    embedding_function_for_chroma = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=EMBEDDING_MODEL_NAME,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    collection = client.get_collection(
        name = COLLECTION_NAME,
        embedding_function = embedding_function_for_chroma
    )
    print(f"Successfully connected to ChromaDB collection '{COLLECTION_NAME}'.")
    print(f"Current count in collection: {collection.count()} chunks.")

except Exception as e:
    print(f"Error connecting to ChromaDB collection: {e}")
    print("Please ensure your 'chroma_db' directory exists and contains data.")
    print("Also verify 'COLLECTION_NAME' and 'EMBEDDING_MODEL_NAME' are correct.")
    # Exit or handle error appropriately if connection fails
    exit()
    


Initializing ChromaDB client at: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data/chroma_db
Successfully connected to ChromaDB collection 'pakistan_laws_chunks_collection'.
Current count in collection: 30122 chunks.


In [6]:
query_embedding_model = None

try:
    query_embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nQuery embedding model '{EMBEDDING_MODEL_NAME}' loaded successfully.")
    
except Exception as e:
    print(f"Error loading query embedding model: {e}")
    print("Please ensure the model can be downloaded/loaded. Exiting for now.")
    exit() # Or handle more gracefully


Query embedding model 'BAAI/bge-large-en-v1.5' loaded successfully.


In [7]:
# --- lets engineer the Retriever Function ---

def retrieve_relevant_chunks(query_text: str, n_results: int = 5) -> list:
    """
    Embeds the query and retrieves the most relevant chunks from ChromaDB.

    Args:
        query_text (str): The user's query.
        n_results (int): The number of top relevant chunks to retrieve.

    Returns:
        list: A list of dictionaries, where each dictionary represents a retrieved chunk
              with its content and metadata.
    """
    if query_embedding_model is None:
        print("Query embedding model not loaded. Cannot retrieve chunks.")
        return []

    # Generate embedding for the query using the same model
    query_embedding = query_embedding_model.encode(query_text).tolist()
    
    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=['documents', 'metadatas', 'distances'] # Request documents, their metadata, and similarity scores
    )
    retrieved_chunks_info = []
    
    if results and results['documents']:
        # Results are lists of lists, so we take the first element (for our single query)
        for i in range(len(results['documents'][0])):
            chunk_content = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i] # Similarity score (lower is more similar)

            retrieved_chunks_info.append({
                "chunk_content": chunk_content,
                "source_file": metadata.get('source_file'),
                "section_title": metadata.get('section_title'),
                "distance": distance # Include distance for debugging/understanding relevance
            })
    
    print(f"\nRetrieved {len(retrieved_chunks_info)} chunks for the query: '{query_text}'")
    return retrieved_chunks_info

In [8]:
# --- Test the Retriever (Optional, but recommended) ---

sample_query = "What are the powers of the Privatisation Commission?"
retrieved_data = retrieve_relevant_chunks(sample_query, n_results=3)
if retrieved_data:
    for i, chunk in enumerate(retrieved_data):
        print(f"\n--- Retrieved Chunk {i+1} (Distance: {chunk['distance']:.4f}) ---")
        print(f"Source: {chunk['source_file']} | Section: {chunk['section_title']}")
        print(f"Content (first 200 chars): {chunk['chunk_content'][:200]}...")
else:
    print("No chunks retrieved.")


    

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Retrieved 3 chunks for the query: 'What are the powers of the Privatisation Commission?'

--- Retrieved Chunk 1 (Distance: 0.4346) ---
Source: administrator00532129aba2e10fe634ab8fbd94c50b.pdf | Section: Preamble/Introduction
Content (first 200 chars): be at Islamabad. The Commission may establish regional offices at such other place or places in Pakistan as it considers necessary. 5. Functions and Powers of the Commission. — The Commission shall: (...

--- Retrieved Chunk 2 (Distance: 0.4857) ---
Source: administrator00532129aba2e10fe634ab8fbd94c50b.pdf | Section: Preamble/Introduction
Content (first 200 chars): THE PRIVATISATION COMMISSION ORDINANCE, 2000 PART I.—GENERAL SECTIONS: 1. Short title, extent and commencement. 2. Definitions. PART II.—PRIVATISATION COMMISSION 3. Establishment of the Commission. 4....

--- Retrieved Chunk 3 (Distance: 0.5315) ---
Source: administrator00532129aba2e10fe634ab8fbd94c50b.pdf | Section: Preamble/Introduction
Content (first 200 chars): 1Subs. by 

In [9]:
# --- Necessary Imports for this Cell ---
# IMPORTANT: Use this import for the latest ChatOllama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- Choose and Initialize the Generative LLM ---
# Connecting to your local Ollama instance running the qwen3:1.7b model.
# Ensure Ollama is running in your terminal (ollama run qwen3:1.7b)
try:
    # --- THIS IS THE CRITICAL LINE ---
    llm = ChatOllama(model="qwen3:1.7b") # <<< Ensure 'model=' is used here
    print("Generative LLM (qwen3:1.7b via Ollama) initialized successfully.")
except Exception as e:
    print(f"Error initializing Ollama LLM: {e}")
    print("Please ensure Ollama is installed and the 'qwen3:1.7b' model is downloaded and running.")
    print("You might need to start Ollama with 'ollama run qwen3:1.7b' in your terminal.")
    exit() # Or handle error more gracefully

Generative LLM (qwen3:1.7b via Ollama) initialized successfully.


In [10]:
from langchain_core.prompts import ChatPromptTemplate
# --- Define the RAG Prompt Template ---
rag_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful legal assistant. Answer the user's question based ONLY on the provided legal context. If the answer is not found in the context, clearly state 'I cannot find a direct answer to your question in the provided documents.'"),
    ("user", "Context: \n{context}\n\nQuestion: {question}")
])
print("\nRAG Prompt Template defined.")


RAG Prompt Template defined.


# # --- Combine Retriever and LLM into a RAG Chain (using LangChain's LCEL) ---

In [11]:
# Define the format_docs function
def format_docs(docs):
    # This line extracts 'chunk_content' from each 'doc' in the 'docs' list
    return "\n\n".join(doc["chunk_content"] for doc in docs)

In [12]:
# The custom_retriever_runnable assumes retrieve_relevant_chunks is defined in Cell 2.
# Make sure Cell 2 was run successfully.
custom_retriever_runnable = RunnablePassthrough.assign(
    context=lambda x: format_docs(retrieve_relevant_chunks(x["question"], n_results=5)) # Adjust n_results as needed
)

In [13]:
rag_chain = custom_retriever_runnable | rag_prompt_template | llm | StrOutputParser()
print("\nRAG chain assembled.")


RAG chain assembled.


In [14]:
# --- Test the Full RAG Pipeline ---
# --- Test the Full RAG Pipeline ---
if __name__ == "__main__":
    print("\n--- Testing the Full RAG Pipeline ---")
    
    # Sample legal query
    question_1 = "What are the functions and powers of the Privatisation Commission as per the Ordinance?"
    print(f"\nQuestion 1: {question_1}")
    response_1 = rag_chain.invoke({"question": question_1})
    print("\nAnswer 1:")
    print(response_1)

    # Another sample query, perhaps one not directly answerable by the documents
    question_2 = "What is the new law regarding cryptocurrency in Pakistan?"
    print(f"\n\nQuestion 2: {question_2}")
    response_2 = rag_chain.invoke({"question": question_2})
    print("\nAnswer 2:")
    print(response_2)


--- Testing the Full RAG Pipeline ---

Question 1: What are the functions and powers of the Privatisation Commission as per the Ordinance?


/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Retrieved 5 chunks for the query: 'What are the functions and powers of the Privatisation Commission as per the Ordinance?'

Answer 1:
<think>
Okay, let's tackle this question. The user is asking about the functions and powers of the Privatisation Commission as per the Ordinance. I need to refer back to the provided context to find the answer.

First, I'll scan through the context. The user mentioned that the context includes parts of the Privatisation Commission Ordinance, 2000, and some amendments. The key sections are Part II, which talks about the functions and powers. 

Looking at the text, under Part II, section 5, it lists the functions and powers. The functions are numbered from (a) to (j). Let me go through each one:

(a) Recommend privatization policy guidelines to the Cabinet.
(b) Prepare a comprehensive privatization programme for Cabinet approval.
(c) Plan, manage, implement, and control the programme.
(d) Prepare and submit reports to the Cabinet on all aspects.
(e) Faci

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Retrieved 5 chunks for the query: 'What is the new law regarding cryptocurrency in Pakistan?'

Answer 2:
<think>
Okay, let's see. The user is asking about the new law regarding cryptocurrency in Pakistan based on the provided context. 

First, I need to go through the context carefully. The context includes several legal acts, like the State Bank of Pakistan (Amendment) Act, 2022, and other related acts. There's a mention of sections 18, 19, 20, and 21, which deal with penalties for publishing laws, offenses, and specific transactions.

Looking at the sections, there's a part about the custody of monies, securities, and other articles, and the sale and realisation of property. Then there's a section about the purchase and sale of securities of countries whose currency is approved foreign exchange. Also, there's a mention of the Federal Government and Provincial Governments purchasing securities.

Wait, the user is asking about cryptocurrency. The context doesn't mention anything about

In [1]:
# --- Necessary Imports for this Cell ---
# IMPORTANT: Use this import for the latest ChatOllama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- Choose and Initialize the Generative LLM ---
# Connecting to your local Ollama instance running the qwen3:1.7b model.
# Ensure Ollama is running in your terminal (ollama run qwen3:1.7b)
try:
    # --- THIS IS THE CRITICAL LINE ---
    llm = ChatOllama(model="qwen3:1.7b") # <<< Ensure 'model=' is used here
    print("Generative LLM (qwen3:1.7b via Ollama) initialized successfully.")
except Exception as e:
    print(f"Error initializing Ollama LLM: {e}")
    print("Please ensure Ollama is installed and the 'qwen3:1.7b' model is downloaded and running.")
    print("You might need to start Ollama with 'ollama run qwen3:1.7b' in your terminal.")
    exit() # Or handle error more gracefully

# --- Define the RAG Prompt Template ---
rag_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful legal assistant. Answer the user's question based ONLY on the provided legal context. If the answer is not found in the context, clearly state 'I cannot find a direct answer to your question in the provided documents.'"),
    ("user", "Context: \n{context}\n\nQuestion: {question}")
])

print("\nRAG Prompt Template defined.")

# --- Combine Retriever and LLM into a RAG Chain (using LangChain's LCEL) ---
# Define the format_docs function
def format_docs(docs):
    # This line extracts 'chunk_content' from each 'doc' in the 'docs' list
    return "\n\n".join(doc["chunk_content"] for doc in docs)

# The custom_retriever_runnable assumes retrieve_relevant_chunks is defined in Cell 2.
# Make sure Cell 2 was run successfully.
custom_retriever_runnable = RunnablePassthrough.assign(
    context=lambda x: format_docs(retrieve_relevant_chunks(x["question"], n_results=5)) # Adjust n_results as needed
)

rag_chain = custom_retriever_runnable | rag_prompt_template | llm | StrOutputParser()

print("\nRAG chain assembled.")

# --- Test the Full RAG Pipeline ---
if __name__ == "__main__":
    print("\n--- Testing the Full RAG Pipeline ---")
    
    # Sample legal query
    question_1 = "What are the functions and powers of the Privatisation Commission as per the Ordinance?"
    print(f"\nQuestion 1: {question_1}")
    response_1 = rag_chain.invoke({"question": question_1})
    print("\nAnswer 1:")
    print(response_1)

    # Another sample query, perhaps one not directly answerable by the documents
    question_2 = "What is the new law regarding cryptocurrency in Pakistan?"
    print(f"\n\nQuestion 2: {question_2}")
    response_2 = rag_chain.invoke({"question": question_2})
    print("\nAnswer 2:")
    print(response_2)

Generative LLM (qwen3:1.7b via Ollama) initialized successfully.

RAG Prompt Template defined.

RAG chain assembled.

--- Testing the Full RAG Pipeline ---

Question 1: What are the functions and powers of the Privatisation Commission as per the Ordinance?


NameError: name 'retrieve_relevant_chunks' is not defined